In [1]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.adatabase import ADatabase
import numpy as np
import pickle

In [2]:
market = Market()
competition_db = ADatabase("competition")

In [3]:
competition_db.connect()
data = competition_db.retrieve("data")
competition_db.disconnect()

In [4]:
market.connect()
sp5 = market.retrieve("sp500")
corrs = market.retrieve("price_corr")
market.disconnect()

In [6]:
start_year = datetime.now().year
end_year = start_year + 1

In [8]:
competition_db.connect()
for training_year in [4]:
    training_corrs = corrs[corrs["training_year"]==training_year]
    for year in tqdm(range(start_year,end_year)):
        for ticker in tqdm(sp5["Symbol"]):
            try:
                if ticker in data.columns:
                    ticker_data = data.copy()
                    example = corrs[(corrs["ticker"]==ticker) & (corrs["year"]==2022) & (corrs["training_year"]==4)].tail(1).fillna(0)[[x for x in corrs.columns if x not in ["ticker","year","training_year"]]].melt()
                    example["abs_corr"] = [abs(x) for x in example["value"]]
                    factors = list(example.sort_values("abs_corr",ascending=False).head(25)["variable"])
                    factors = [x for x in factors if x in ticker_data.columns]
                    training_data = ticker_data[(ticker_data["year"]<year) & (ticker_data["year"]>=year-training_year)]
                    training_data["y"] = training_data[ticker].shift(-1)
                    training_data.dropna(inplace=True)
                    training_data.reset_index(drop=True,inplace=True)
                    prediction_data = ticker_data[ticker_data["year"]==year]
                    if training_data.index.size > 100:
                        refined_data = {"X":training_data[factors],"y":training_data[["y"]]}
                        models = m.regression(refined_data)
                        models["year"] = year
                        models["training_year"] = training_year
                        models["ticker"] = ticker
                        models["model"] = [pickle.dumps(x) for x in models["model"]]
                        competition_db.store("models",models)
                    else:
                        continue
                else:
                    continue
            except Exception as e:
                print(year,ticker,str(e))
competition_db.disconnect()

 11%|█████████████▏                                                                                                             | 54/504 [04:36<34:47,  4.64s/it]


 22%|███████████████████████████▎                                                                                              | 113/504 [09:17<37:06,  5.70s/it]


 33%|████████████████████████████████████████▋                                                                                 | 168/504 [14:08<29:26,  5.26s/it]


 45%|██████████████████████████████████████████████████████▍                                                                   | 225/504 [18:59<26:16,  5.65s/it]


 56%|████████████████████████████████████████████████████████████████████▋                                                     | 284/504 [23:47<18:44,  5.11s/it]


 68%|███████████████████████████████████████████████████████████████████████████████████                                       | 343/504 [28:43<12:19,  4.59s/it]


 79%|████████████████████████████████████████████████████████████████████████████████████████████████▊                         | 400/504 [33:25<09:53,  5.71s/it]


 90%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████▍            | 452/504 [38:09<05:05,  5.88s/it]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [41:53<00:00, 2513.25s/it]
